In [174]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from selenium_stealth import stealth

In [175]:
scraped_links=pd.read_csv("scholarships_pending.csv").sort_values("Unnamed: 0",ascending=True).iloc[:,1:]
scraped_links.head()

,Unnamed: 0,Beca,Tipo de ayuda,Universidad,Ubicación,Beneficios,Plazo,Enlace
0,0,Merit Scholarships,Merit-based,Temple University,"Philadelphia, United States",2000 USD,2000 USD,https://www.mastersportal.com/scholarships/171...
1,1,MRS Resident Advisor Scholarship,Merit-based,Monash University,"Clayton, Australia",565 AUD,565 AUD,https://www.mastersportal.com/scholarships/206...
2,2,CROUS Bursary,Need-based,University of Strasbourg,"Strasbourg, France",Various benefits,Various benefits,https://www.mastersportal.com/scholarships/101...
3,3,Chevening Scholarships,Merit-based,University of Bedfordshire,"Luton, United Kingdom",Various benefits,Various benefits,https://www.mastersportal.com/scholarships/929...
4,4,British Chevening Scholarships,Merit-based,Cranfield University,"Cranfield, United Kingdom",Various benefits,Various benefits,https://www.mastersportal.com/scholarships/910...


In [176]:
# Especificar la ruta del ChromeDriver descargado
chrome_driver_path = "C:\\Users\\edwin\\chromedriver-win64\\chromedriver.exe"

chrome_options = webdriver.ChromeOptions()

# Añadir un User-Agent personalizado
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")

# Usar el Service de Selenium para especificar la ruta
driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win64",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

In [177]:
file_path='scholarships_pending.csv'
output_file = 'scholarship_additional_info_2.csv'

In [180]:
# Crear el archivo CSV para guardar los datos extraídos
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # Escribir la cabecera
    writer.writerow(['ID','URL','About', 'OrganisationInformation', 'Overview', 'Eligibility', 'Application'])

    # Leer el archivo CSV con los links
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        # Saltar la primera fila que contiene los headers
        next(reader)
        for row in reader:
            id_file=row[0]
            print(id_file)
            url = row[8]  # Asumiendo que cada fila tiene un link
            
            # Iniciar el navegador y cargar la página
            driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)
            driver.get(url)
            time.sleep(3)  # Espera para que cargue la página
            try: about=driver.find_element(By.ID, 'AboutSection').text
            except Exception as e:
                print(f"Error en about section: {e}")

            # Cerrar pop-up
            try:
                # Encontrar el contenedor padre del botón de cerrar
                close_button_container = driver.find_element(By.CLASS_NAME, 'SoftWallCloseButton')
                
                # Desplazar a la vista
                driver.execute_script("arguments[0].scrollIntoView(true);", close_button_container)

                # Hacer clic usando JavaScript
                driver.execute_script("arguments[0].click();", close_button_container)
                print("Botón de cerrar clicado mediante el contenedor.")
                time.sleep(3)
            except Exception as e:
                print(f"Error: {e}")


            # Inicializar variables para almacenar la información
            overview_section = eligibility_section = application_section = "N/A"

            # Extraer información de la sección "Overview"
            try:
                overview_section = driver.find_element(By.XPATH, '//*[@data-id="overview"]').text
            except Exception as e:
                print(f"Error extrayendo Overview: {e}")
            
            # Extraer información de la sección "OrgInformation"
            try:
                org_information = driver.find_element(By.ID, 'OrganisationInformation').text
            except Exception as e:
                print(f"Error extrayendo OrgInformation: {e}")


            # Extraer información de la sección "Elegibility"
            try:

                eligibility_button = driver.find_element(By.XPATH, '//button[@class="ContentButton js-contentButtons js-cypressEligibilityButton" and @data-id="eligibility"]')

                eligibility_button.click()

                # Hacer clic usando JavaScript
                driver.execute_script("arguments[0].click();", eligibility_button)
                print("Botón 'Eligibility' clicado con éxito.")
                
                time.sleep(3)
                
                #Extrae elegibility
                eligibility_section=driver.find_element(By.XPATH, '//*[@data-id="eligibility"]').text

            except Exception as e:
                print(f"Error: {e}")

            # Cambiar a la sección "Application" y extraer información
            try:
                application_button = driver.find_element(By.XPATH, '//button[@class="ContentButton js-contentButtons js-cypressApplicationButton" and @data-id="application"]')

                application_button.click()

                # Hacer clic usando JavaScript
                driver.execute_script("arguments[0].click();", application_button)
                print("Botón 'Application' clicado con éxito.")

                time.sleep(3)  # Espera para que cambie la sección
                application_section = driver.find_element(By.XPATH, '//*[@data-id="application"]').text
                
            except Exception as e:
                print(f"Error extrayendo Application: {e}")

            # Escribir los datos en el archivo CSV
            writer.writerow([id_file, url,   about, overview_section, eligibility_section, application_section, org_information])

            # Cerrar el navegador después de procesar el enlace
            driver.quit()


0
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
1
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
2
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
3
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
4
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
5
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
6
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Application' clicado con éxito.
7
Botón de cerrar clicado mediante el contenedor.
Botón 'Eligibility' clicado con éxito.
Botón 'Applicat

In [183]:
final_df=pd.read_csv('scholarship_additional_info_2.csv')
final_df.tail()

,ID,URL,About,OrganisationInformation,Overview,Eligibility,Application
1562,1672,https://www.mastersportal.com/scholarships/902...,About\nAalto University has a scholarship prog...,Overview\nScholarship type\nMerit based\nNumbe...,Eligibility\nThe Incentive-Based Scholarship c...,Application\nApplication deadline\nAnytime\nTo...,"Aalto University\nHelsinki , Finland\nNot rank..."
1563,1673,https://www.mastersportal.com/scholarships/889...,About\nSSA offers two scholarship options avai...,Overview\nScholarship type\nMerit based\nNumbe...,Eligibility\nThis scholarship is especially de...,Application\nApplication deadline\n15 May 2025...,"Aalto University\nHelsinki , Finland\nNot rank..."
1564,1674,https://www.mastersportal.com/scholarships/452...,About\nAudencia Business School wants to make ...,Overview\nScholarship type\nMerit and need bas...,Eligibility\nThe scholarships are awarded in t...,Application\nApplication deadline\n01 Mar 2025,"Audencia\nNantes , France\nNot ranked\nStudypo..."
1565,1675,https://www.mastersportal.com/scholarships/761...,About\nCentral European University is an under...,Overview\nScholarship type\nMerit and need bas...,Eligibility\nEnrolled students who wish to att...,Application\nApplication deadline\n10 Nov 2024...,"Central European University (CEU)\nVienna , Au..."
1566,2460,https://www.mastersportal.com/scholarships/475...,About\nBall State University can support you f...,Overview\nScholarship type\nMerit and need bas...,"Eligibility\nBall State graduate students, or ...",Application\nApplication deadline\nNot specifi...,"Ball State University\nMuncie , United States\..."


In [184]:
final_df.shape

(1567, 7)